In [ ]:
import pandas as pd
import numpy as np

# Load data
ratings = pd.read_csv('epinions_rating.txt', sep=' ', names=['user', 'item', 'rating', 'timestamp'])
trust = pd.read_csv('epinions_trust.txt', sep=' ', names=['user', 'trusted_user'])

# Create user-item interaction matrix
user_item_matrix = ratings.pivot(index='user', columns='item', values='rating').fillna(0)


In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, n_users, n_items, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(n_users + n_items, 128),
            nn.ReLU(),
            nn.Linear(128, 1),  # Predict trust-weighted score
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, n_users, n_items):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(n_users + n_items, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
def train_gan(generator, discriminator, data_loader, epochs, g_optimizer, d_optimizer, criterion):
    for epoch in range(epochs):
        for real_data in data_loader:
            batch_size = real_data.size(0)

            # Train discriminator
            real_labels = torch.ones(batch_size, 1)
            fake_labels = torch.zeros(batch_size, 1)

            d_optimizer.zero_grad()
            real_outputs = discriminator(real_data)
            d_loss_real = criterion(real_outputs, real_labels)

            z = torch.randn(batch_size, real_data.size(1))
            fake_data = generator(z)
            fake_outputs = discriminator(fake_data.detach())
            d_loss_fake = criterion(fake_outputs, fake_labels)

            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()

            # Train generator
            g_optimizer.zero_grad()
            fake_outputs = discriminator(fake_data)
            g_loss = criterion(fake_outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')
